In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
import ipywidgets as widgets

import os
import subprocess

%matplotlib notebook

In [11]:
root_folder = '/home/alessio/Desktop/CONICAL2_3/d-FromScan_around23/1-c-other-around-all_space'
lab='allGeoms'
subfolders = sorted([dir for dir in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder,dir))])
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> zNorbornadiene_N001-500_P016-615_P092-458
1 -> zNorbornadiene_N002-000_P016-308_P092-458
2 -> zNorbornadiene_N002-000_P016-615_P092-011
3 -> zNorbornadiene_N002-000_P016-615_P092-905
4 -> zNorbornadiene_N002-000_P016-923_P092-458
5 -> zNorbornadiene_N002-500_P016-615_P092-458
6 -> zNorbornadiene_P000-000_P015-077_P092-458
7 -> zNorbornadiene_P000-500_P015-077_P092-458



In [12]:
list_of_greps = []

for i,fol in enumerate(subfolders):
    geom_fol = os.path.join(root_folder,fol,lab)
    command = 'grep norm: ' + geom_fol + "/*/*.out | awk '{print $3}'"
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    numpy_array = np.array([float(x) for x in output.split(b'\n') if x != b''])
    list_of_greps.append(numpy_array)

In [13]:
def plot_histogramZ(list_of_greps,high,resol,normalize):

    bins = np.arange(0,high,resol)

    fig, ax = plt.subplots(figsize=(15, 6))
    _, bins, patches = plt.hist([ np.clip(x, bins[0], bins[-1]) for x in list_of_greps ],
                                #density=normalize,  # <- normalization 
                                bins=bins, 
                                #color=['#3782CC', '#AFD5FA', '#000000'], 
                                label=subfolders)

    xlabels = bins[1:].astype(str)
    xlabels[-1] += '+'

    N_labels = len(xlabels)
    plt.xlim([0, high])
    plt.xticks(resol * np.arange(N_labels) + (resol/2))
    ax.set_xticklabels(xlabels)

    #plt.yticks([])
    plt.title(r'Distribution of values for Derivative coupling between $S_1$ / $S_2$')
    plt.setp(patches, linewidth=0)
    plt.legend(loc='upper right')

    fig.tight_layout()

def wrapper(maximum_bin,resolution,normalize):
    plot_histogramZ(list_of_greps, maximum_bin, resolution,normalize)
    
widgets.interact(wrapper, 
                 maximum_bin = widgets.FloatText(value=10), 
                 resolution = widgets.FloatText(value=1),
                 normalize = widgets.Checkbox(value=True, description='Normalize'),);

interactive(children=(FloatText(value=10.0, description='maximum_bin'), FloatText(value=1.0, description='reso…